# chunker: default program

In [1]:
from default import *
import os

## Run the default solution on dev

In [4]:
chunker = LSTMTagger(os.path.join('data', 'train.txt.gz'), os.path.join('data', 'chunker'), '.tar')
decoder_output = chunker.decode('data/input/dev.txt')

100%|██████████| 1027/1027 [00:02<00:00, 459.66it/s]


## Evaluate the default output

In [5]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 11672 phrases; correct: 8568.
accuracy:  84.35%; (non-O)
accuracy:  85.65%; precision:  73.41%; recall:  72.02%; FB1:  72.71
             ADJP: precision:  36.49%; recall:  11.95%; FB1:  18.00  74
             ADVP: precision:  71.36%; recall:  39.45%; FB1:  50.81  220
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  70.33%; recall:  76.80%; FB1:  73.42  6811
               PP: precision:  92.40%; recall:  87.14%; FB1:  89.69  2302
              PRT: precision:  65.00%; recall:  57.78%; FB1:  61.18  40
             SBAR: precision:  84.62%; recall:  41.77%; FB1:  55.93  117
               VP: precision:  63.66%; recall:  58.25%; FB1:  60.83  2108


(73.40644276901988, 72.02420981842637, 72.70875763747455)

## Documentation

This documentation contains different methods. 

### Baseline Method 1
final score 75.9. lower than method2, so we didn't spend much time on improve this code.</br>
In below code, we concatenates the word embedding with the char-level-representation. </br>
This code is written in LSTMTaggerModel.forward</br>

In [ ]:
import string

embeds = self.word_embeddings(sentence)

char_level_rep = []
for w_ix in sentence:
    word = self.ix_to_word[w_ix.item()]
    v1 = [0]*100
    v2 = [0]*100
    v3 = [0]*100
    v1[string.printable.index(word[0])] = 1
    v3[string.printable.index(word[-1])] = 1
    if(len(word)>2):
        for i in range(1,len(word)):
            ind = string.printable.index(word[i])
            v2[ind] = v2[ind]+1
    char_level_rep.append(v1+v2+v3)
char_level_rep = torch.tensor(char_level_rep)

embeds = torch.cat([embeds, char_level_rep], dim=1)

### Evaluate the method1 output

processed 23663 tokens with 11896 phrases; found: 11836 phrases; correct: 9010.</br>
accuracy:  86.14%; (non-O)</br>
accuracy:  87.24%; precision:  76.12%; recall:  75.74%; FB1:  75.93</br>
             ADJP: precision:  40.00%; recall:  15.93%; FB1:  22.78  90</br>
             ADVP: precision:  68.98%; recall:  47.49%; FB1:  56.25  274</br>
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0</br>
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0</br>
               NP: precision:  73.84%; recall:  79.41%; FB1:  76.52  6708</br>
               PP: precision:  91.91%; recall:  87.96%; FB1:  89.89  2336</br>
              PRT: precision:  58.70%; recall:  60.00%; FB1:  59.34  46</br>
             SBAR: precision:  82.35%; recall:  47.26%; FB1:  60.05  136</br>
               VP: precision:  68.83%; recall:  67.10%; FB1:  67.96  2246</br>
(76.1236904359581, 75.73974445191661, 75.93123209169053)


### Baseline Method 2
final score 75.1337. </br>
In below code, we used second RNN to encode char-level-representation, and concatenated the result of 64dim encoding to word embeddings as input of chunker RNN.</br>
This code is written in LSTMTaggerModel.forward</br>

In [ ]:
import string

embeds = self.word_embeddings(sentence)
char_level_rep = []
for w_ix in sentence:
    word = self.ix_to_word[w_ix.item()]
    v1 = [0.]*100
    v2 = [0.]*100
    v3 = [0.]*100
    v1[string.printable.index(word[0])] = 1.
    v3[string.printable.index(word[-1])] = 1.
    #if(len(word)>2):
    for i in range(0,len(word)):
        ind = string.printable.index(word[i])
        v2[ind] = v2[ind]+1
    char_level_rep.append(v1+v2+v3)
char_level_rep = torch.tensor(char_level_rep)
        
clr_lstm_out, _ = self.clr_lstm(char_level_rep.view(len(sentence), 1, -1))
clr_lstm_out = clr_lstm_out.reshape((clr_lstm_out.shape[0],self.hidden_dim))
embeds = torch.cat([embeds, clr_lstm_out], dim=1)

### Evaluate the method2 output

processed 23663 tokens with 11896 phrases; found: 11843 phrases; correct: 8918.</br>
accuracy:  85.61%; (non-O)</br>
accuracy:  86.80%; precision:  75.30%; recall:  74.97%; FB1:  75.13</br>
             ADJP: precision:  47.13%; recall:  18.14%; FB1:  26.20  87</br>
             ADVP: precision:  68.15%; recall:  42.46%; FB1:  52.32  248</br>
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0</br>
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0</br>
               NP: precision:  72.54%; recall:  79.22%; FB1:  75.74  6811</br>
               PP: precision:  91.57%; recall:  86.28%; FB1:  88.84  2300</br>
              PRT: precision:  74.29%; recall:  57.78%; FB1:  65.00  35</br>
             SBAR: precision:  86.96%; recall:  42.19%; FB1:  56.82  115</br>
               VP: precision:  68.31%; recall:  66.62%; FB1:  67.46  2247</br>
dev.out score: 75.1337

## Analysis

We found that increasing of accuracy of (non-O) improves the entire accuracy of the program; therefore, we focused on fixing misspelled data.

## Approaches
1. make new vector 'v4' for contain vowls</br>
2. use smoothing with weights unknow: +1 and known: +10</br>

### Baseline Method 2 with approaches
final score 76.9848. </br>
In below code, we implemented approaches above on the method2.</br>
This code is written in LSTMTaggerModel.forward</br>

In [ ]:
import string
embeds = self.word_embeddings(sentence)
char_level_rep = []
vowls= ['a','e','i','o','u']
for w_ix in sentence:
    word = self.ix_to_word[w_ix.item()]
    v1 = [1.]*100
    v2 = [1.]*100
    v3 = [1.]*100
    v4 = [1.]*100
    v1[string.printable.index(word[0])] = 10.
    v3[string.printable.index(word[-1])] = 10.
    #if(len(word)>2):
    for i in range(0,len(word)):
        ind = string.printable.index(word[i])
        if i in vowls:
            v2[ind] = v2[ind]+10
        else:
            v4[ind] = v4[ind]+10
    char_level_rep.append(v1+v2+v3+v4)
char_level_rep = torch.tensor(char_level_rep)
        
clr_lstm_out, _ = self.clr_lstm(char_level_rep.view(len(sentence), 1, -1))
clr_lstm_out = clr_lstm_out.reshape((clr_lstm_out.shape[0],self.hidden_dim))
embeds = torch.cat([embeds, clr_lstm_out], dim=1)

### Evaluate the method2 output

processed 23663 tokens with 11896 phrases; found: 12036 phrases; correct: 9212. </br>
accuracy:  86.61%; (non-O)</br>
accuracy:  87.76%; precision:  76.54%; recall:  77.44%; FB1:  76.98</br>
             ADJP: precision:  41.07%; recall:  20.35%; FB1:  27.22  112</br>
             ADVP: precision:  68.73%; recall:  50.25%; FB1:  58.06  291</br>
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0</br>
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0</br>
               NP: precision:  75.00%; recall:  81.05%; FB1:  77.91  6740</br>
               PP: precision:  91.65%; recall:  88.16%; FB1:  89.87  2348</br>
              PRT: precision:  64.86%; recall:  53.33%; FB1:  58.54  37</br>
             SBAR: precision:  78.91%; recall:  42.62%; FB1:  55.34  128</br>
               VP: precision:  68.66%; recall:  70.92%; FB1:  69.77  2380</br>
dev.out score: 76.9848